In [ ]:
!nvidia-smi

Tue Oct  1 13:43:11 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off | 00000000:00:04.0 Off |                    0 |
| N/A   32C    P0              46W / 400W |      2MiB / 40960MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

In [ ]:
df

In [ ]:
path = '/content/drive/MyDrive/DACON/Finance/reprocessed/'
path = '/content/drive/MyDrive/kdt-EST-AI/project/dacon_fis/src/'
base_directory = path # Your Base Directory

In [ ]:
dataset_name = "kdt3/DACON-QA-large-ensemble-markdown-reprocessed-256"
train_name = "kdt3/DACON-QA-large-ensemble-markdown-refactored-256"
fname = "gemma2_large_ensemble_markdown_256_3epoch_refactored.csv"

# 설명

## Question - Answering with Retrieval

본 대회의 과제는 중앙정부 재정 정보에 대한 **검색 기능**을 개선하고 활용도를 높이는 질의응답 알고리즘을 개발하는 것입니다. <br>이를 통해 방대한 재정 데이터를 일반 국민과 전문가 모두가 쉽게 접근하고 활용할 수 있도록 하는 것이 목표입니다. <br><br>
베이스라인에서는 평가 데이터셋만을 활용하여 source pdf 마다 Vector DB를 구축한 뒤 langchain 라이브러리와 llama-2-ko-7b 모델을 사용하여 RAG 프로세스를 통해 추론하는 과정을 담고 있습니다. <br>( train_set을 활용한 훈련 과정은 포함하지 않으며, test_set  에 대한 추론만 진행합니다. )

## Mount/Login

구글 드라이브를 마운트하고 허깅페이스에 로그인
- 이때 허깅페이스 토큰은 kdt3 그룹에 대해 읽기/쓰기 권한이 있는 토큰이어야 함

## Download Library
필요/사용 라이브러리 다운로드
이때 버전 문제로 설치를 한 뒤 세션을 한번 재시작해줘야 합니다
<br>(그리고 세션 완전히 끊기면 다운로드 후 재시작을 다시 해줘야...)

## Import Library
한번 재시작했으면 위 과정 없이 Import만 실행해주면 됩니다

## Vector DB
문서를 여러 조각(chunk)로 나누고, 임베딩 유사도를 통해 관련 조각을 찾을 수 있게 DB화하는 함수들이 정의되어 있습니다.

## DB 생성
Vector DB에서 정의된 함수들로 문서 DB를 만들어줍니다.<br><br>
이때 Train과 Test를 한번에 하려고 하면 코랩이 터질 확률이 높으므로 Train하고 Create Dataset까지 실행해 업로드 한 뒤 재시작해서 램을 비우고 Test를 하는 것이 좋습니다.<br> 또한 문서 임베딩을 어떤 모델로 할지 인자로 넘겨줄 수 있습니다

## Create Dataset
DB 생성에서 만든 db와 데이터 dataframe을 사용해 HuggingFace 데이터셋 생성 후 업로드

## Fine-Tuning
학습 데이터셋으로 모델에 대한 파인튜닝 진행 후 Huggingface에 업로드<br>
4비트 양자화 LoRA로 파인튜닝<br>
기반 모델 또는 넣어줄때 사용할 프롬프트, 학습 관련 하이퍼파라미터 수정 가능

## Langchain 을 이용한 추론
모델을 사용한 추론


## 실행
### 기본
Mount/Login -> Download Library -> 재시작 (처음 1번)
Mount/Login -> Import Library (이후)

### 데이터셋 만들기
기본 -> Vector DB -> DB 생성 -> Create Dataset에서 첫 셀 + Train/Valid/Test 중 해당하는 셀

### 모델 학습하기
기본 -> Fine-Tuning(업로드할 위치, 데이터셋 위치, 모델 링크 확인 필수)

### 학습된 모델로 추론하기
기본 -> Langchain을 이용한 추론(모델 링크, 데이터셋 위치 확인) -> Submission(저장할 파일명 확인)

# Mount/Login

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os

token_path = os.path.join(base_directory,'data','token')
with open(token_path,'r') as f:
    hf_token = f.readline().strip('\n')
    wandb_token = f.readline().strip('\n')

In [ ]:
from huggingface_hub import login

login(token=hf_token, add_to_git_credential=True)

Token is valid (permission: fineGrained).
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
!pip install wandb
import wandb

# wandb 개인 API 키 입력
wandb.login(key=wandb_token)

import os

os.environ["WANDB_PROJECT"]="DACON_FinAI"
os.environ["WANDB_LOG_MODEL"] = "checkpoint"
wandb_run_name="noaug-Llama3.1-lora-base"

# Download Library

In [ ]:
#!pip install unstructured pdfminer.six
#!pip install pillow-heif
#!pip install unstructured_inference
#!pip install unstructured_pytesseract
#!pip install pikepdf pypdf
#!pip install PyMuPDF

In [ ]:
!apt-get install tesseract-ocr
!apt-get install poppler-utils

!pip install orjson==3.10.6

!pip install accelerate
!pip install -i https://pypi.org/simple/ bitsandbytes
!pip install transformers[torch] -U

!pip install datasets
!pip install langchain
!pip install langchain_community
!pip install langchain-teddynote
!pip install sentence-transformers
!pip install faiss-gpu
!pip install peft
!pip install trl

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
poppler-utils is already the newest version (22.02.0-2ubuntu0.5).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.
Looking in indexes: https://pypi.org/simple/


# Import Library

In [ ]:
import os
import unicodedata

import torch
import pandas as pd
from tqdm.auto import tqdm

from langchain.document_loaders.parsers.pdf import PDFPlumberParser


from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    pipeline,
    BitsAndBytesConfig
)
from accelerate import Accelerator

# peft
from peft import prepare_model_for_kbit_training
from peft import PeftModel
from peft import LoraConfig, get_peft_model


# Langchain 관련
from langchain.llms import HuggingFacePipeline
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.schema import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter, MarkdownHeaderTextSplitter
from langchain.prompts import PromptTemplate
from langchain.schema.runnable import RunnablePassthrough, RunnableParallel
from langchain.schema.output_parser import StrOutputParser

# PDF 로딩/청크화 관련
from langchain.document_loaders.parsers.pdf import PDFPlumberParser
from langchain.document_loaders.pdf import PDFPlumberLoader
from langchain.document_loaders import UnstructuredPDFLoader
from langchain_teddynote.retrievers import KiwiBM25Retriever
from langchain.retrievers import EnsembleRetriever, MultiQueryRetriever


/usr/local/lib/python3.10/dist-packages/langchain_teddynote/retrievers/__init__.py:14: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  from .kiwi_bm25 import KiwiBM25Retriever


# Fine-Tuning

In [ ]:
# 모델 ID
model_cands={
 'llama2' : "beomi/llama-2-ko-7b",
 'yi' : "beomi/Yi-Ko-6B",
 'solar-beom' : "beomi/Solar-Ko-Recovery-11B",
 'gemma2' : "rtzr/ko-gemma-2-9b-it",
 'solar-lee' : "chihoonlee10/T3Q-ko-solar-dpo-v8.0",
 'llama3' : "KISTI-KONI/KONI-Llama3-8B-Instruct-20240729",
}

model_id = model_cands['gemma2']

In [ ]:
# 모델 로드 및 양자화 설정 적용

def load_model_w_setting(model_id,add_output_token=False,**kwargs):
  # 4비트 양자화 설정
  bnb_config = BitsAndBytesConfig(
      load_in_4bit=True,
      bnb_4bit_use_double_quant=True,
      bnb_4bit_quant_type="nf4",
      bnb_4bit_compute_dtype=torch.bfloat16
  )

  # 토크나이저 로드 및 설정
  tokenizer = AutoTokenizer.from_pretrained(model_id)
  tokenizer.use_default_system_prompt = False
  tokenizer.padding_side="right"

  model= AutoModelForCausalLM.from_pretrained(
      model_id,
      quantization_config=bnb_config,
      device_map="auto",
      trust_remote_code=True,
      **kwargs
      )

# 일부 모델의 경우 토크나이저에 답변 토큰 추가 작업 필요
  if add_output_token :
    initial_token_count = len(tokenizer)
    response_template = '답변: '
    added_token_count = tokenizer.add_special_tokens({"additional_special_tokens": [response_template]})
    model.resize_token_embeddings(new_num_tokens=initial_token_count+added_token_count)

  return model, tokenizer

In [ ]:
model,tokenizer = load_model_w_setting(model_id,attn_implementation='eager')

Loading checkpoint shards:   0%|          | 0/10 [00:00<?, ?it/s]

In [ ]:
#데이터셋 로드
from datasets import load_dataset

# dataset_url = "kdt3/DACON-QA-base-table-preprocessed-v3"
# dataset_url = "kdt3/DACON-QA-base-preprocessed-v2"
# dataset_url = "kdt3/DACON-QA-base-augselect"
# dataset_url = "kdt3/DACON-QA-base-markdown"
# dataset_url = "kdt3/DACON-QA-bge-markdown"
dataset_url = dataset_name


train_dataset = load_dataset(dataset_url)

In [ ]:
# 문맥 잘못됐나 확인 - 문맥 길이 체크
import numpy as np
amax = np.argmax([len(x) for x in train_dataset['train']['context']])
amax, len(train_dataset['train']['context'][amax])

(77, 1955)

In [ ]:
def print_dataset_ele(data,i):
  print(data['question'][i])
  print('--------')
  print(data['context'][i])
  print('--------')
  print(data['answer'][i])

#예시
i= amax
print_dataset_ele(train_dataset['train'],i)

예산 증감률이 가장 높은 분야는 무엇인가요?
--------
예산개요  
-----  
y g  
제
1  
##### 보건 · 복지 · 고용 분야 부  
## 01  
1-1 재정지원 방향  
2024년 보건·복지·고용분야 재정투자 규모는 242.9조원 수준이며, 총지출 대비 비중이  
37.0%로 12대 분야 중 가장 비중이 크다. 소득·고용·주거 안전망에 대한 투자  
2024년도 예산안 시정연설 및 제안설명  
확충으로 저소득·노인·장애인 등 취약계층을 더욱 두텁게 지원하고, 가족돌봄청년 등
0.9%이다.  
[표 3-2] 2024년 예산지출 규모  
(단위: 조원, %)  
|(nan, '구 (nan, '202|3년 (nan, '|2024년 (nan, '|증감률 \n(B-|
|---|---|---|---|  
분') **\n(A)')** **\n(B)')** **A)/A')**  
0 ■ 예산지출 441.1 438.3 △0.6  
1 •일반회계 369.4 356.5 △3.5  
2 •특별회계 71.6 81.7 14.1
근무복 점퍼도 보온력이 높은 구스 재질로 고품질화하는 한편, 시설관리, 청사방호,  
견위생, 통역 등 공무직 인원 증원도 추진할 예정이다.  
-----  
0 Summary of Budget for FY 2024  
[표 11-4] 경찰 부문 주요 변동내역  
(단위: 억원, %)  
0 합 계 128,064 133,486 4.2  
|구 분 2 (|023년 2 a) (|024년 b)|증감률 ((b-a|)/ 비 고 a)|
계획이다.  
-----  
0 2024년 나라살림예산개  
0 2024  
[표 6-5] 항공·공항 부문 주요 변동내역  
|구 분 2023년 (|b) 2024년 (b|) 증감률 ((b-|a)/a)|
|---|---|---|---|  
1 공항건설 및 관리 1,601 7,374 360.5  
2 항공발전, 운영지원 1,834 1,532 △16.4  
|Col1|Col2|Col3|Col4|
|---|---|---|-

In [ ]:
from peft import LoraConfig, get_peft_model

config = LoraConfig(
    r=16,
    lora_alpha=64,
    target_modules=[
        "q_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
        "lm_head",
    ],
    bias="none",
    lora_dropout=0.05,
    task_type="CAUSAL_LM",
)

model = get_peft_model(model, config)

/usr/local/lib/python3.10/dist-packages/peft/tuners/tuners_utils.py:498: UserWarning: Model with `tie_word_embeddings=True` and the tied_target_modules=['lm_head'] are part of the adapter. This can lead to complications, for example when merging the adapter or converting your model to formats other than safetensors. See for example https://github.com/huggingface/peft/issues/2018.
  warnings.warn(


In [ ]:
# 학습 프롬프트 - 추론 프롬프트와 통일하는 것이 좋다고 함
template = """
task_instructions" : [

 당신은 재정 정보 관련 전문가 입니다. 문서를 바탕으로 질문에 한 문장 이내로 답변하세요.
 1. 문서에 있는 내용을 자르거나 편집하지 않고 그대로 가져오세요.
 2. 순서에 따른 번호를 매기지 마세요. 출력 시 불이익을 줄 것입니다.
 3. 수치에 단위가 있다면 문서를 바탕으로 답변에 단위를 포함하세요.
 4. 질문의 키워드를 바탕으로 문서를 끝까지 검토하세요.
 5. 한 단어 혹은 단어의 나열이 아닌, 완성된 한국어 문장으로 답변하세요.
 6. 답변 외에 예시, 참고, 정보 출처, 신뢰도, 확장된 답변, '답변: ', '참고: '를 절대로 출력하지 마세요.

]

"context":
{context},

"question":
{question},

"주어진 질문에 대한 답변만 한 문장으로 생성한다."

"answer":
{answer}
"""

response_template = "'answer':\n"

def formatting_prompts_func(example, template=template):
    output_texts = []
    for i in range(len(example['question'])):
        context = example['context'][i]
        question = example['question'][i]
        answer = example['answer'][i]
        output_texts.append(template.format(context=context,question=question,answer=answer))
    return output_texts



In [ ]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, roc_auc_score
import collections
# from SQUAD, 메트릭 계산 함수

def compute_f1(y_true, y_pred):
    common = collections.Counter(y_true) & collections.Counter(y_pred)
    num_same = sum(common.values())
    precision = 1.0 * num_same / len(y_pred) if len(y_pred) != 0 else 0
    recall = 1.0 * num_same / len(y_true) if len(y_true) != 0 else 0
    if len(y_pred) == 0 or len(y_true) == 0: f1 = int(y_pred == y_true)
    elif num_same == 0: f1 = 0
    else: f1 = (2 * precision * recall) / (precision + recall)
    return f1, precision, recall

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    f1, precision, recall = compute_f1(labels,preds)
#    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    acc = accuracy_score(labels, preds)
    auc = roc_auc_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall,
        'auroc': auc
    }

In [ ]:
import transformers, os
from datetime import datetime
from trl import SFTConfig, SFTTrainer, DataCollatorForCompletionOnlyLM

# 로컬에 모델 저장하고 싶은 경우 이름 지정
project = "financeQA-finetune"
base_model_name = "gemma2"
run_name = base_model_name + "_" + project
output_dir = os.path.join(path,run_name)
if not os.path.exists(output_dir) : os.makedirs(output_dir)

MAX_LEN = 2048

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=train_dataset['train'],
    eval_dataset = train_dataset['valid'],
    compute_metrics=compute_metrics,
    args=transformers.TrainingArguments(
        do_eval=True,
        output_dir=output_dir,
        warmup_ratio=0.05,
        per_device_train_batch_size=1,
        gradient_accumulation_steps=4,
        gradient_checkpointing=True,
        num_train_epochs = 3,
        learning_rate=2e-4,
        fp16=True,
        optim="paged_adamw_8bit",
        logging_strategy='epoch',
        report_to="wandb",
        run_name=wandb_run_name,
        # save_strategy="steps",
        # save_steps=25,

    ),
    max_seq_length=MAX_LEN,
    formatting_func=formatting_prompts_func, # 프롬프트 처리하기 위해 필요
    data_collator=DataCollatorForCompletionOnlyLM(response_template, tokenizer=tokenizer, mlm=False), #모델에게 답변생성에만 집중하도록 함
)

model.config.use_cache = False
trainer.train()

trained_model = (trainer.model.module if hasattr(trainer.model, "module") else trainer.model)

#로컬에 저장할 경우
# trained_model.save_pretrained(f"{output_dir}/saved_model")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': max_seq_length. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:283: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(


Map:   0%|          | 0/496 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)
/usr/local/lib/python3.10/dist-packages/trl/trainer/utils.py:156: UserWarning: Could not find response key `'answer':
` in the following instance: <bos>
task_instructions" : [

 당신은 재정 정보 관련 전문가 입니다. 문서를 바탕으로 질문에 한 문장 이내로 답변하세요.
 1. 문서에 있는 내용을 자르거나 편집하지 않고 그대로 가져오세요.
 2. 순서에 따른 번호를 매기지 마세요. 출력 시 불이익을 줄 것입니다.
 3. 수치에 단위가 있다면 문서를 바탕으로 답변에 단위를 포함하세요.
 4. 질문의 키워드를 바탕으로 문서를 끝까지 검토하세요.
 5. 한 단어 혹은 단어의 나열이 아닌, 완성된 한국어 문장으로 답변하세요.
 6. 답변 외에 예시, 참고, 정보 출처, 신뢰도, 확장된 답변, '답변: ', '참고: '를 절대로 출력하지 마세요.

]

"context":
`60. 진실･화해를위한과거사정리위원회` - ············································· `335`  
-----  
##### Ⅰ 성과계획서 개요  
1. 성과계획서 의의  
(1) 성과계획서 의의  
□ 성과계획서는중앙관서및기금관리주체의전략목표및성과목표를  
달성하기위한연도별시행계획  
□ 성과계획서는사업실시전예산편성단계에서성과목표및성과지표,  
목표수준을사전에설정ㆍ관리
```
* 성과

Step,Training Loss
124,0.000000
248,0.000000
372,0.000000


스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
□ 2024년도성과계획서작성대상 60개중앙관서의성과관리체계는전략  
목표 187개, 프로그램목표 475개로구성
|2|사업은 * 성과관리 대상에서 제외기 준 ▪정부연구소 인건비, 출연(연) 운영지원 비목만으로 구성된 사업|  
-----  
2. 성과지표 및 목표치 설정  
(1) 성과지표 설정  
□ 성과지표는프로그램목표가추구하는목적달성여부를측정하는도구로서  
대표성과포괄성을유지하도록설정  
ㅇ 목표달성정도를정확히측정하고지속적인평가를통해성과수준의  
변화를시계열로파악할수있도록설정  
□ 성과지표는가급적국민이체감할수있는궁극적성과에대한결과·  
정량지표로설정
```
2024년도

# 성과계획서
(총괄편)

## 대한민국정부
|16|57 새 만 금 개|발|청|3|3 3|
|17|58 해 양 경 찰||청|5|5 5|
|18|59 5·18 민주|진상 규명||1|1 1|
||화운 동|조사 위원||||
|||회||||
|19|60 진 실 · 화 해 를 위 한 거 정 과 사|리 원 위|회|1|1 1|  
-----  
2. 성과지표 유형  
□ 2024년도프로그램목표 475개의성과지표는총 507개로프로그램목표당
,

"question":
2024년 성과지표의 결과지표는 몇 개로 구성되어 있나요?,

"주어진 질문에 대한 답변만 한 문장으로 생성한다."

"answer":
성과지표의 성격에 따라 결과지표는 355개로 구성되어 있습니다.
 This instance will be ignored in loss calculation. Note, if this happens often, consider increasing the `max_seq_length`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/utils.py:156: UserWarning: Could not find response key `'answer':
` in the foll

In [ ]:
# 모델 업로드
trained_model.push_to_hub(train_name, private=True,save_embedding_layers=True)

/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:227: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")


adapter_model.safetensors:   0%|          | 0.00/2.05G [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/kdt3/DACON-QA-large-ensemble-markdown-refactored-256/commit/92761bbc69e0739542cb509d467143b11b94a7f3', commit_message='Upload model', commit_description='', oid='92761bbc69e0739542cb509d467143b11b94a7f3', pr_url=None, pr_revision=None, pr_num=None)

# Langchain 을 이용한 추론

In [ ]:
# gpu memory 할당 해제
import gc
torch.cuda.empty_cache()
gc.collect()

18

In [ ]:
# HuggingFacePipeline 객체 생성

# 모델 ID
model_id = model_cands[run_name.split('_')[0]]
peft_model_id = train_name
trained_model,tokenizer = load_model_w_setting(model_id)

#Fine-Tune 한 LoRA 어댑터 불러오기
trained_model.load_adapter(peft_model_id)

text_generation_pipeline = pipeline(
    model=trained_model,
    tokenizer=tokenizer,
    task="text-generation",
    return_full_text=False,
    max_new_tokens=200,
    # repetition_penalty=1.5,
    eos_token_id = tokenizer.eos_token_id,
    pad_token_id = tokenizer.pad_token_id,
    max_length=MAX_LEN
)

llm = HuggingFacePipeline(pipeline=text_generation_pipeline)

Loading checkpoint shards:   0%|          | 0/10 [00:00<?, ?it/s]

adapter_config.json:   0%|          | 0.00/725 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/peft/tuners/tuners_utils.py:498: UserWarning: Model with `tie_word_embeddings=True` and the tied_target_modules=['lm_head'] are part of the adapter. This can lead to complications, for example when merging the adapter or converting your model to formats other than safetensors. See for example https://github.com/huggingface/peft/issues/2018.
  warnings.warn(


adapter_model.safetensors:   0%|          | 0.00/2.05G [00:00<?, ?B/s]

<ipython-input-25-a5e29cb1a3c2>:45: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=text_generation_pipeline)


In [ ]:
#데이터셋 로드
from datasets import load_dataset
dataset_url = dataset_name
dataset = load_dataset(dataset_url)

In [ ]:
# 검증 데이터 쓸지, 테스트 데이터 쓸지
eval_mode = 'valid' # or 'test'
eval_dataset = dataset[eval_mode]

In [ ]:
# 그냥 GPU 메모리 확인용
!nvidia-smi

Tue Oct  1 15:01:23 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off | 00000000:00:04.0 Off |                    0 |
| N/A   32C    P0              52W / 400W |  15199MiB / 40960MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

In [ ]:
template[:-9]

'\ntask_instructions" : [\n\n 당신은 재정 정보 관련 전문가 입니다. 문서를 바탕으로 질문에 한 문장 이내로 답변하세요.\n 1. 문서에 있는 내용을 자르거나 편집하지 않고 그대로 가져오세요.\n 2. 순서에 따른 번호를 매기지 마세요. 출력 시 불이익을 줄 것입니다.\n 3. 수치에 단위가 있다면 문서를 바탕으로 답변에 단위를 포함하세요.\n 4. 질문의 키워드를 바탕으로 문서를 끝까지 검토하세요.\n 5. 한 단어 혹은 단어의 나열이 아닌, 완성된 한국어 문장으로 답변하세요.\n 6. 답변 외에 예시, 참고, 정보 출처, 신뢰도, 확장된 답변, \'답변: \', \'참고: \'를 절대로 출력하지 마세요.\n\n]\n\n"context":\n{context},\n\n"question":\n{question},\n\n"주어진 질문에 대한 답변만 한 문장으로 생성한다."\n\n"answer":\n'

In [ ]:
from tqdm.auto import trange

# 결과를 저장할 리스트 초기화
results = []

# DATASET 구조를 dataset[i]={'question':,'context':,...}로 바꾸면 안됨?

# Dataset 각 행에 대해 처리
for idx in trange(len(eval_dataset['question']), desc="Answering Questions"):
    #질문, 컨텍스트(문서)
    question = eval_dataset['question'][idx]
    context = eval_dataset['context'][idx]

    # RAG 체인 구성
    prompt = PromptTemplate.from_template(template[:-9])

    # RAG 체인 정의
    if context != "":
        rag_chain = (
            RunnableParallel(context=lambda x: x["context"], question = lambda x: x["question"])
            | prompt
            | llm
            | StrOutputParser()
        )
    else:
        rag_chain = (
            {"question": RunnablePassthrough()}
            | prompt
            | llm
            | StrOutputParser()
        )

    # 답변 추론
    print(f"Question: {question}")
    full_response = rag_chain.invoke({"question": question, "context": context})

    print(f"Answer: {full_response}\n")

    # 결과 저장

    if context != "":
        results.append({
            "Context": context,
            "Question": question,
            "Answer": full_response,
            "True_Answer": eval_dataset['answer'][idx]
        })
    else:
        results.append({
            "Question": question,
            "Answer": full_response,
            "True_Answer": eval_dataset['answer'][idx]
        })

Answering Questions:   0%|          | 0/100 [00:00<?, ?it/s]

Both `max_new_tokens` (=200) and `max_length`(=2048) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Question: 2024년 중앙정부 재정체계는 어떻게 구성되어 있나요?


Both `max_new_tokens` (=200) and `max_length`(=2048) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Answer: 2024년 기준으로 중앙정부 재정체계는 일반회계 1개, 특별회계 21개, 기금 68개로 구성되어 있습니다.

Question: 2024년 중앙정부의 예산 지출은 어떻게 구성되어 있나요?


...

Answer: 2024년 농림·수산·식품 분야 재정투자 규모는 2023년 24.4조원 대비 1조원, 4.3% 증가하여 25.4조원이다.

Question: 임업인의 소득안정과 공익적 기능 강화를 위해 어떠한 지원책이 시행되고 있는가?


Both `max_new_tokens` (=200) and `max_length`(=2048) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Answer: 임업인의 소득안정 및 임업의 공익적 기능을 강화하기 위한 임업직불금(566억원 → 620억원), 임업인 정책자금 지원을 위한 산림사업종합자금(이차보전)(+29억원)을 지속적으로 지원한다." 


Question: 하수관로정비 사업 예산은 2023년 대비 2024년에 얼마나 증가했는가?


Both `max_new_tokens` (=200) and `max_length`(=2048) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Answer: 하수관로정비 사업 예산은 2023년 대비 2024년에 3조 285 억원으로 증가했습니다.

Question: 국방부가 장병 복무여건을 획기적으로 개선하기 위해 어떤 조치를 취하고 있는가?
Answer: 국방부는 병 봉급을 월 125만원으로, 사회진출지원금을 월 40만원으로 인상하고, 지역상생장병특식을 매월 1회씩 그리고 호국보훈의 달과 국군의 날에 1회씩 시행하도록 확대하며, 예비군 훈련시설을 현대화하고, 초급간부의 단기복무 장려금을 인상하는 등의 조치를 취하고 있습니다.  




In [ ]:
i = 28
print_dataset_ele(eval_dataset,i)

2023년 에너지바우처 사업 예산에서 사업운영비 중 시스템 고도화에 얼마가 할당되었나요?
--------
-  대상자 확대 등에 따른 홍보, 시스템 고도화, 콜센터 운영 비용 증액  
-  (산출) 에너지복지 홍보(448백만원), 시스템 고도화(705백만원), 콜센터 운영(502.7백만원), 패널조사  
(131.7백만원), 전달체계 구축(34.6백만원), 주택관리공단 운영지원(50백만원)  
ㅇ 2023년도 예산 및 2024년도 예산 산출 세부내역 비교
업운영비 (1,349백만원) ㆍ 에너지복 0.33만 세대 × 31만원 = 1,023백만원
지 홍보: 328백만원 ㆍ 시스템 고도 마. 사업운영비 (1,872백만원) ㆍ 에너지
화 : 520백만원 ㆍ 콜센터 운영 : 280 복지 홍보: 448백만원 ㆍ 시스템 고도화 :
백만원 ㆍ 패널조사 : 133.5백만원 705백만원 ㆍ 콜센터 운영 : 502.7백만원
ㆍ 에너지바우처 전달체계 구축 : ㆍ 패널조사 : 131.7백만원 ㆍ 에너지바우
-「새정부 에너지정책 방향(국무회의 의결)」을 통해 필수 에너지복지 확대[*] 발표(’22.7)  
-  (에너지바우처) 에너지바우처 지원 확대를 통해 에너지복지 사각지대 해소 추진  
-  2023년도 에너지바우처 사업 시행(‘23.5~’23.12월 신청·접수, ‘23.7~’24.4월 사용)  
□ 주요내용  
① 사업규모  
-  총사업비(해당되는 경우에만 기재) : 해당없음  
-  사업기간 : ’15년 ~ 계속
## 9) 최근 3년간 동 사업에 대한 주요 외부지적사항 및 평가, 문제점 및 대책  
□ ‘21년 국정감사 지적사항  
ㅇ 저소득층의 폭염 피해 최소화를 위해 동절기 에너지바우처 일부를 하절기에 사용  
할 수 있도록 제도 개선할 것  
- 희망세대 대상으로 동절기 에너지바우처 중 일부를 하절기에 사용할 수 있도록  
시스템 개선하였음(‘22.5월)  
□ ‘21년 결산 지적사항
백만원 라. 등유바우처 (1,023백만원) ㆍ 0.33만 세대 × 31만원 = 1,02

# Evaluation

In [ ]:
#검증 데이터 쓸때만 사용 가능
if eval_mode == 'valid':
  for item in results:
      y_hat = item["Answer"]
      y = item["True_Answer"]
      f1, precision, recall = compute_f1(y, y_hat)
      item["F1"] = f1
      item["Precision"] = precision
      item["Recall"] = recall

In [ ]:
if eval_mode == 'valid':
  # 제출용 샘플 파일 로드
  eval_df = pd.DataFrame([])

  # 생성된 답변을 제출 DataFrame에 추가
  eval_df['Question'] = [item['Question'] for item in results]
  eval_df['Answer'] = [item['Answer'] for item in results]
  eval_df["F1"] = [item["F1"] for item in results]
  eval_df["Precision"] = [item["Precision"] for item in results]
  eval_df["Recall"] = [item["Recall"] for item in results]
  # eval_df['Answer'] = eval_df['Answer'].fillna("데이콘")     # 모델에서 빈 값 (NaN) 생성 시 채점에 오류가 날 수 있음 [ 주의 ]

  save_dir = os.path.join(path,'eval')
  if not os.path.exists(save_dir) : os.makedirs(save_dir)
  save_name = f'eval_{fname}'
  save_path = os.path.join(save_dir,save_name)

  # 결과를 CSV 파일로 저장
  eval_df.to_csv(save_path, encoding='UTF-8-sig', index=False)

In [ ]:
# 평균 F1 확인
# eval_df = pd.read_csv(f"{path}trained_eval.csv",index_col=0)
if eval_mode == 'valid' :
  display(eval_df["F1"].mean())

0.7124401948554958

# Submission

In [ ]:
submit_df = pd.read_csv(f"{path}sample_submission.csv")
submit_df.head()

,SAMPLE_ID,Answer
0,TEST_000,데이콘
1,TEST_001,데이콘
2,TEST_002,데이콘
3,TEST_003,데이콘
4,TEST_004,데이콘


In [ ]:
# 제출용 샘플 파일 로드
submit_df = pd.read_csv(f"{path}sample_submission.csv")

# 생성된 답변을 제출 DataFrame에 추가
save_mode = 'submission'

if save_mode != 'submission' :
  submit_df['Question'] = [item['Question'] for item in results]
  submit_df['Context'] = [item['Context'] for item in results]
  save_dir = os.path.join(path,'eval')
else : save_dir = os.path.join(path,'sub')

if not os.path.exists(save_dir) : os.makedirs(save_dir)
save_path = os.path.join(save_dir,fname)

submit_df['Answer'] = [item['Answer'] for item in results]
submit_df['Answer'] = submit_df['Answer'].fillna("데이콘").apply(str.rstrip)     # 모델에서 빈 값 (NaN) 생성 시 채점에 오류가 날 수 있음 [ 주의 ]

# 결과를 CSV 파일로 저장
submit_df.to_csv(save_path, encoding='UTF-8-sig', index=False)